# Understanding why people LOVE & HATE your brand or product.

## Table of Content:
1. Set Algorithmia & Twitter API Keys
2. Search Twitter for a certain keyword.
3. Run Social Sentiment Analysis on tweets
4. Extract topics from tweets using LDA

# 1. Set Algorithmia & Twitter API Keys

In [1]:
import Algorithmia
import pprint
from twython import Twython, TwythonError

In [2]:
Algorithmia.apiKey = 'simxxxxxxx'

APP_KEY = 'xxxxxxx'
APP_SECRET = 'xxxxxxx'
OAUTH_TOKEN = 'xxxxxxx'
OAUTH_TOKEN_SECRET = 'xxxxxxx'

input = {"query":"comcast","numTweets":500}

pp = pprint.PrettyPrinter(indent=2)

# 2. Search Twitter for a certain keyword.

In [3]:
auth = {'app_key': APP_KEY, 'app_secret': APP_SECRET, 'oauth_token': OAUTH_TOKEN, 'oauth_token_secret': OAUTH_TOKEN_SECRET}
twitter_input = {'query': input['query'], 'numTweets': int(input['numTweets']), 'auth': auth}

all_tweets = Algorithmia.algo('twitter/RetrieveTweetsWithKeyword/0.1.0').pipe(twitter_input)

# 3. Run Social Sentiment Analysis on tweets

In [4]:
tweet_reponses = []
for tweet in all_tweets:
    try:
        tweet_reponses.append(Algorithmia.algo('nlp/SocialSentimentAnalysis/0.1.2').pipe(tweet)[0])
    except Exception as e:
        pass

tweet_compound = map(lambda x: {'sentence': x['sentence'], 'compound': x['compound']}, tweet_reponses)

tweet_compound.sort(key=lambda x: x['compound'], reverse=True)

In [5]:
coeff = int(0.2 * len(tweet_compound))

# Get top 20% (positive) of tweets and disclude neutral tweets if needed
pos_tweets = tweet_compound[:coeff]
pos_tweets = filter(lambda x: float(x['compound']) != 0.0, pos_tweets)
pos_tweets = map(lambda x: x['sentence'], pos_tweets)

# Get bottom 20% (negative) of tweets and disclude neutral tweets if needed
neg_tweets = tweet_compound[-coeff:]
neg_tweets = filter(lambda x: float(x['compound']) != 0.0, neg_tweets)
neg_tweets = map(lambda x: x['sentence'], neg_tweets)

# 4. Extract topics from tweets using LDA

In [6]:
# Get LDA for positive tweets
pos_input = {'docsList': pos_tweets, 'mode': 'smart'}
pos_lda = Algorithmia.algo('nlp/LDA/0.1.0').pipe(pos_input)

# Get LDA for negative tweets
neg_input = {'docsList': neg_tweets, 'mode': 'smart'}
neg_lda = Algorithmia.algo('nlp/LDA/0.1.0').pipe(neg_input)

In [7]:
pp.pprint(pos_lda)

[ { u'account': 3,
    u'awarded': 2,
    u'comcast': 4,
    u'details': 3,
    u'experience': 3,
    u'house': 2,
    u'https://t.co': 3,
    u'liz': 3},
  { u'awesome': 6,
    u'blizzard': 6,
    u'comcast': 15,
    u'hackathon': 4,
    u'pennapps': 4,
    u'ride': 4,
    u'snowy': 4,
    u'weekend': 6},
  { u'comcast': 23,
    u'customer': 5,
    u'good': 4,
    u'great': 6,
    u'https://t.co/mrljt': 3,
    u'internet': 10,
    u'service': 5,
    u'zfcc': 3},
  { u'american': 16,
    u'comcast': 21,
    u'great': 17,
    u'hsgshrtl': 15,
    u'https://t.co/j': 15,
    u'nra': 32,
    u'phillymag': 16,
    u'show': 17}]


In [8]:
pp.pprint(neg_lda)

[ { u'annoying': 3,
    u'bad': 5,
    u'comcast': 24,
    u'customer': 5,
    u'home': 3,
    u'service': 13,
    u'waiting': 4,
    u'worst': 4},
  { u'comcast': 23,
    u'customers': 11,
    u'disabled': 11,
    u'disconnect': 11,
    u'hardship': 11,
    u'medical': 11,
    u'physically': 11,
    u'protection': 11},
  { u'actress': 13,
    u'james': 13,
    u'l.a': 11,
    u'lily': 13,
    u'pose': 13,
    u'prejudice': 13,
    u'pride': 13,
    u'strikes': 13},
  { u'america': 3,
    u'comcast': 16,
    u'fix': 3,
    u'fuck': 3,
    u'hate': 4,
    u'nra': 6,
    u'phone': 3,
    u'shit': 6}]
